In [ ]:
import pandas as pd
from scipy.stats import ttest_ind


def cargar_y_procesar(ruta_csv):
    df = pd.read_csv(ruta_csv)
    df['fecha'] = pd.to_datetime(df['fecha'])
    
    # Clima 
    df['clima_adverso'] = ((df['clima'] == 'Lluvioso') | (df['clima'] == 'Húmedo')).astype(int)
    
    # Sueño 
    mapeo_sueno = {
        'Horrible': 1,
        'Malo': 2,
        'Ok': 3,
        'Bueno': 4,
        'Excelente': 5
    }
    df['sueno_score'] = df['sueño'].replace(mapeo_sueno)
    
    # Dieta 
    mapeo_dieta = {
        'Inflamatoria': 0,
        'Balanceada': 1,
        'Antiinflamatoria': 2
    }
    df['dieta_score'] = df['dieta_tipo'].replace(mapeo_dieta)
    
    # Calcular promedio de los síntomas
    columnas_sintomas = ['dolor_score', 'inflamacion_score', 'fatiga_score', 'rigidez_score']
    df['ISA'] = df[columnas_sintomas].mean(axis=1)
    
    # DataFrame numérico
    columnas_numericas = [
        'temperatura_C', 'actividad_fisica', 'sueno_score', 'dieta_score',
        'clima_adverso', 'en_brote', 'rigidez_score', 'dolor_score',
        'inflamacion_score', 'fatiga_score', 'estado_animo_score', 'ISA'
    ]
    df_numerico = df[columnas_numericas]
    
    # Matriz de correlación
    matriz_correlacion = df_numerico.corr()
    
    # Función para obtener correlaciones
    def obtener_correlaciones(columna):
        return matriz_correlacion[columna].sort_values(ascending=False)
    
    correlaciones_dolor = obtener_correlaciones('dolor_score')
    correlaciones_fatiga = obtener_correlaciones('fatiga_score')
    correlaciones_animo = obtener_correlaciones('estado_animo_score')
    correlaciones_ISA = obtener_correlaciones('ISA')

    # Correlación Sueño vs Fatiga 
    sueno_T1 = df['sueno_score'].shift(1).dropna()
    fatiga_T = df['fatiga_score'].iloc[1:]
    
    # Calcular correlación 
    prom_sueno = sueno_T1.mean()
    prom_fatiga = fatiga_T.mean()
    
    diferencias_sueno = sueno_T1.values - prom_sueno
    diferencias_fatiga = fatiga_T.values - prom_fatiga
    
    cov = (diferencias_sueno * diferencias_fatiga).sum()
    var_sueno = (diferencias_sueno ** 2).sum()
    var_fatiga = (diferencias_fatiga ** 2).sum()
    
    correlacion_sueno_fatiga = cov / ((var_sueno * var_fatiga) ** 0.5)

    # Prueba t para clima adverso
    dolor_clima_adverso = df[df['clima_adverso'] == 1]['dolor_score']
    dolor_clima_normal = df[df['clima_adverso'] == 0]['dolor_score']
    
    t_stat, p_val = ttest_ind(dolor_clima_adverso, dolor_clima_normal)
    resultado_t_clima = {'t_stat': t_stat, 'p_val': p_val}

    return (df, df_numerico, matriz_correlacion, correlaciones_dolor, 
            correlaciones_fatiga, correlaciones_animo, correlaciones_ISA,
            resultado_t_clima, correlacion_sueno_fatiga)